In [ ]:
# Szükséges könyvtárak importálása
import numpy as np                  # Numerikus műveletekhez (tömbök, stb.)
import matplotlib.pyplot as plt     # Ábrázoláshoz
import os                           # Operációs rendszerrel kapcsolatos funkciók (pl. könyvtárkezelés)
import shutil                       # Fájl műveletek (pl. könyvtár törlése)
import time                         # Időméréshez (a szimuláció futási idejének mérésére)
from scipy.spatial import KDTree    # A sűrűség becsléséhez (K-D fa a legközelebbi szomszédok kereséséhez)

# Szimulációs paraméterek ---
N_PARTICLES = 150000          # Részecskék száma
TIME_STEPS = 90               # Szimulációs lépések teljes száma
DT = 0.01                     # Időlépés mérete
THETA = 0.7                   # Barnes-Hut nyitási szög paraméter (0.7 egy gyakori érték)
G = 1.0                       # Gravitációs állandó (egyszerűsített egységekben)
PLOT_EVERY = 1                # Mentsen egy képkockát minden N lépésben
OUTPUT_DIR = "nbody_szimulacio_kimenet" # Kimeneti könyvtár a képkockák mentéséhez
FRAME_PADDING_DIGITS = 5      # A képkocka fájlnevek számjegyeinek száma (pl. frame_00001.png)
K_NEIGHBORS_FOR_DENSITY = 20  # Hány legközelebbi szomszédot használjunk a sűrűségbecsléshez

# Kimeneti könyvtár létrehozása, ha nem létezik
if os.path.exists(OUTPUT_DIR):  # Létezik-e ez a mappa
    shutil.rmtree(OUTPUT_DIR)   # Régi képkockák és mappa eltávolítása, ha a könyvtár létezik
os.makedirs(OUTPUT_DIR)         # Új mappa elkészítése
print(f"A kimeneti képkockák ide lesznek mentve: {OUTPUT_DIR}")

# Részecske inicializálás ---
# Részecskék véletlenszerű inicializálása egy korong vagy gömbszerű eloszlásban
# Az egyszerűség kedvéért itt 2D korong eloszlást használunk.

# Véletlen szögek és sugarak generálása
angles = np.random.rand(N_PARTICLES) * 2 * np.pi # [0,1)-ből egyenletesen * 2pi
# Kerüljük a pontos középpontot, illetve gyököt vonunk, így területileg "egyenletes"
radii = np.sqrt(np.random.rand(N_PARTICLES)) * 0.4 + 0.05

# Pozíciók (x, y)
pos = np.zeros((N_PARTICLES, 2))
pos[:, 0] = radii * np.cos(angles) # első oszlop
pos[:, 1] = radii * np.sin(angles) # második oszlop

# (vx, vy) a kezdeti keringési mozgás
vel = np.zeros((N_PARTICLES, 2))
# Egyszerű körsebesség közelítés: v arányos sqrt(G*M/r), M ~ r^2 -> v ~ sqrt(r)
# Érintő irány: (-y, x)
vel[:, 0] = -0.5 * np.sqrt(radii) * np.sin(angles)
vel[:, 1] =  0.5 * np.sqrt(radii) * np.cos(angles)

# Az egyszerűség kedvéért minden részecske tömege azonos
mass = np.ones(N_PARTICLES) / N_PARTICLES # Teljes tömeg = 1

# QuadTree
class QuadTreeNode:
    def __init__(self, center, size): #konstruktor
        self.center = center        # node régiójának középpont koordinátái (x, y) np.array
        self.size = size            # node négyzet alakú régiójának szélessége/magassága
        self.children = [None] * 4  # Mutatók a gyerek nodeokra (ÉK, ÉNy, DNy, DK)
        self.particle_indices = []  # Részecskék indexei, amelyeket közvetlenül ez a node tartalmaz (ha levél)
        self.is_leaf = True         # Ez a node levél node?
        self.total_mass = 0.0       # A nodeon belüli részecskék teljes tömege
        self.center_of_mass = np.zeros(2) # A nodeon belüli részecskék tömegközéppontja

    # Meghatározza, melyik kvadránsba tartozik egy részecske
    def get_quadrant(self, particle_pos):
        if particle_pos[0] >= self.center[0]: # Kelet
            if particle_pos[1] >= self.center[1]: # Észak-Kelet
                return 0
            else: # Dél-Kelet
                return 3
        else: # Nyugat
            if particle_pos[1] >= self.center[1]: # Észak-Nyugat
                return 1
            else: # Dél-Nyugat
                return 2

    # Rekurzívan beszúr egy részecske indexet a fába
    def insert(self, p_idx, particle_positions):
        # Ha ez a node levél
        if self.is_leaf:
            # Ha üres, csak adjuk hozzá a részecskét
            if not self.particle_indices:
                self.particle_indices.append(p_idx)
            else:
                # Ha már tartalmaz részecskét, fel kell osztani
                self.is_leaf = False # Többé nem levél
                # Gyerek nodeok létrehozása
                half_size = self.size / 2.0
                centers = [
                    [self.center[0] + half_size/2, self.center[1] + half_size/2], # ÉK
                    [self.center[0] - half_size/2, self.center[1] + half_size/2], # ÉNy
                    [self.center[0] - half_size/2, self.center[1] - half_size/2], # DNy
                    [self.center[0] + half_size/2, self.center[1] - half_size/2]  # DK
                ]
                for i in range(4):
                  # 4 gyerek node létrhozása
                    self.children[i] = QuadTreeNode(np.array(centers[i]), half_size)

                # A meglévő részecske áthelyezése a megfelelő gyerekbe
                # törli az eddig ebben a nodeban tárolt részecske indexét.
                existing_p_idx = self.particle_indices.pop()
                # régi részecske melyik új gyerek-kvadránsba esik
                quadrant_existing = self.get_quadrant(particle_positions[existing_p_idx])
                # rekurzívan meghívja az insert-öt, hogy a régi részecskét elhelyezze
                self.children[quadrant_existing].insert(existing_p_idx, particle_positions)

                # Az új részecske beszúrása a megfelelő gyerekbe
                # új részecske melyik új gyerek-kvadránsba esik
                quadrant_new = self.get_quadrant(particle_positions[p_idx])
                # rekurzívan meghívja az insert-öt, hogy az új részecskét elhelyezze
                self.children[quadrant_new].insert(p_idx, particle_positions)
        else:
            # Ha nem levél, szúrjuk be a megfelelő gyerekbe
            quadrant = self.get_quadrant(particle_positions[p_idx])
            # Lehetséges, hogy egy részecske pont a határon van, ilyenkor próbálkozunk a következővel,
            # de ez ritka és itt most nem kezeljük expliciten a lebegőpontos pontatlanságokat.
            # Ha a gyerek None (nem kellene előfordulnia a logika szerint), hibát jelezhetünk.
            if self.children[quadrant] is None:
                 # Ez a helyzet nem fordulhatna elő, ha a felosztás helyes volt.
                 print(f"Hiba: Üres gyerek node a beszúrásnál, kvadráns {quadrant}, részecske {p_idx}")
                 # Fallback: próbáljuk meg a gyökér középpontja alapján, bár ez nem ideális
                 half_size = self.size / 2.0
                 centers = [
                    [self.center[0] + half_size/2, self.center[1] + half_size/2], # ÉK
                    [self.center[0] - half_size/2, self.center[1] + half_size/2], # ÉNy
                    [self.center[0] - half_size/2, self.center[1] - half_size/2], # DNy
                    [self.center[0] + half_size/2, self.center[1] - half_size/2]  # DK
                 ]
                 self.children[quadrant] = QuadTreeNode(np.array(centers[quadrant]), half_size)
            # rekurzívan meghívjuk az insert-öt de már egy kisebb részproblémán
            # ez addig megy, amíg el nem jut egy levélbe, ahol már tudja,
            # hogy mit kell csinálnia
            self.children[quadrant].insert(p_idx, particle_positions)


    # Kiszámítja a tömegközéppontját ennek a nodenak és rekurzívan a gyerekeknek
    def compute_mass_distribution(self, particle_positions, particle_masses):
        if self.is_leaf:
            if self.particle_indices:
                p_idx = self.particle_indices[0]
                # ennek a pontnak a súlya és helye
                self.total_mass = particle_masses[p_idx]
                self.center_of_mass = particle_positions[p_idx]
            else:
              # ha üres a levél
                self.total_mass = 0.0
                self.center_of_mass = np.zeros(2)
        else:
            self.total_mass = 0.0
            self.center_of_mass = np.zeros(2)
            # végigmegy a node gyerekein
            for child in self.children:
                if child:
                    # rekurzívan meghívja magát, így a fa aljától számolunk mindent
                    child.compute_mass_distribution(particle_positions, particle_masses)
                    self.total_mass += child.total_mass
                    # Súlyozott átlag a tömegközépponthoz: sum(m_i * r_i) / sum(m_i)
                    # ez most csak a sum(m_i * r_i)
                    self.center_of_mass += child.total_mass * child.center_of_mass

            if self.total_mass > 1e-9: # Nullával való osztás elkerülése, ha a node üres
                self.center_of_mass /= self.total_mass
            else:
                 # Ha nincs tömeg, alapértelmezett a geometriai középpont
                 # Bár ez a helyzet nem ideális, a számítás folytatódhat
                 # A szülő nodeok helyesen kezelik a 0 tömegű gyerekeket,
                 # Azaz nem lesz gravitációs vonzereje.
                 self.center_of_mass = self.center


# QuadTree Építése ---
def build_quadtree(particle_positions, particle_masses):
    # Az összes részecskét tartalmazó határoló doboz megkeresése
    # Biztonsági ellenőrzés: vannak-e részecskék?
    if len(particle_positions) == 0:
         # Visszaadhatunk egy üres gyökeret vagy hibát dobhatunk
         print("Nincsenek részecskék a QuadTree építéséhez.")
         # Egy alapértelmezett, üres gyökér visszaadása
         return QuadTreeNode(np.array([0.0, 0.0]), 1.0) # Vagy más alapértelmezett érték

    min_coord = np.min(particle_positions, axis=0) # axis=0 miatt oszloponként (koordinátánként) keresi a minumumot
    max_coord = np.max(particle_positions, axis=0)
    center = (min_coord + max_coord) / 2.0
    size_vec = max_coord - min_coord
    # Kezeljük azt az esetet, ha minden részecske egy pontban vagy egy vonalon van
    size = np.max(size_vec)
    if size < 1e-9: # Ha a méret gyakorlatilag nulla
        size = 1.0 # Adjunk neki egy minimális méretet
    size *= 1.1 # Adjunk hozzá egy kis puffert, így a határon levők is bent lesznek

    # Gyökér node létrehozása
    root = QuadTreeNode(center, size)

    # Összes részecske beszúrása
    for i in range(len(particle_positions)):
        # Biztosítsuk, hogy a részecske a gyökér határain belül van.
        # Ez általában teljesül a számítás módja miatt, de egy extra ellenőrzés nem árt.
        # Ha egy részecske kívül esik, az problémát okozhat a kvadráns meghatározásában.
        # Itt most feltételezzük, hogy minden részecske a számított határokon belül van.
        try:
            root.insert(i, particle_positions)
        except Exception as e:
            print(f"Hiba a(z) {i}. részecske beszúrása közben: {e}")
            print(f"Részecske pozíciója: {particle_positions[i]}, Fa középpont: {root.center}, Fa méret: {root.size}")
            # Dönthetünk úgy, hogy kihagyjuk a részecskét, vagy leállítjuk a szimulációt
            continue # Kihagyjuk ezt a részecskét és folytatjuk


    # Tömegeloszlások kiszámítása (minden node-ra)
    root.compute_mass_distribution(particle_positions, particle_masses)
    return root

# Erőszámítás (Barnes-Hut) ---
def calculate_force_on_particle(p_idx, node, particle_positions, particle_masses, theta):
    # Kiszámítja a 'p_idx' indexű részecskére ható gravitációs erőt,
    # amelyet a 'node' nodeban lévő részecskék fejtenek ki.
    force = np.zeros(2)
    pos_p = particle_positions[p_idx]

    # Ha a nodenak nincs tömege, nincs erőhatás tőle
    if node.total_mass < 1e-9:
        return force

    # Vektor a p részecskétől a node tömegközéppontjáig
    dr = node.center_of_mass - pos_p
    dist_sq = np.sum(dr**2)

    # Adjunk hozzá egy kis lágyítási tényezőt (softening) a nullával való osztás
    # és a túl nagy erők elkerülése érdekében közeli kölcsönhatásoknál.
    softening_sq = 0.01**2 # Ezt a paramétert hangolni lehet
    dist = np.sqrt(dist_sq + softening_sq)

    # Ellenőrizzük a távolságot is, hogy ne legyen túl kicsi
    if dist < 1e-6: # Kerüljük a numerikus instabilitást
         return force # Ha túl közel van, ne számoljunk erőt (vagy használjunk csak lágyítást)

    # Barnes-Hut kritérium: s/d < theta
    # s = a node szélessége (node.size)
    # d = távolság a p részecskétől a node tömegközéppontjáig (dist)
    criterion = node.size / dist

    if node.is_leaf:
        # Ha levél node, és van benne részecske, ami nem maga p_idx
        if node.particle_indices and node.particle_indices[0] != p_idx:
            # Közvetlen számítás: F = G * m1 * m2 / r^2 * (r_vec / r)
            # A tömegek már a particle_masses tömbben vannak, node.total_mass itt a levélben lévő egyetlen részecske tömege
            force_magnitude = G * particle_masses[p_idx] * node.total_mass / (dist_sq + softening_sq)
            force = force_magnitude * dr / dist
        # Ha a levél üres vagy önmagát tartalmazza, az erő 0 (a ciklus elején ezt már kezeltük a dist < 1e-6 ellenőrzéssel is)

    elif criterion < theta:
        # Ha a node elég messze van (vagy külső node), kezeljük egyetlen ponttömegként
        # node.total_mass > 0 már ellenőrizve lett a függvény elején
        force_magnitude = G * particle_masses[p_idx] * node.total_mass / (dist_sq + softening_sq)
        force = force_magnitude * dr / dist
    else:
        # A node túl közel van, figyelembe kell venni a gyerekeit
        for child in node.children:
            # Csak akkor menjünk le, ha a gyerek létezik és van tömege
            if child and child.total_mass > 1e-9:
                # rekurzívan meghívja önmagát a gyerekre, a kapott erővektort pedig
                # hozzáadja a p_idx részecskére ható teljes erőhöz, tehát
                # az erő a gyerek node-tól származó erők összege lesz
                force += calculate_force_on_particle(p_idx, child, particle_positions, particle_masses, theta)

    return force


# Összes Erő Kiszámítása ---
def get_all_forces(root, particle_positions, particle_masses, theta):
    n = len(particle_positions)
    forces = np.zeros((n, 2))
    for i in range(n):
        # Kiszámítjuk az i-edik részecskére ható erőt a fa gyökerétől indulva
        forces[i] = calculate_force_on_particle(i, root, particle_positions, particle_masses, theta)
    # minden részecskére tartalmazza a rá ható teljes erővektort
    return forces

# Időintegrálás (Leapfrog - egyszerűsített) ---
# A Leapfrog (szökkenő béka) módszer jó gravitációs szimulációkhoz, mert jól megőrzi az energiát.
# Lépések egy teljes Leapfrog implementációban:
# 1. Sebességek frissítése fél lépéssel: v_{i+1/2} = v_i + a_i * dt/2
# 2. Pozíciók frissítése egész lépéssel: x_{i+1} = x_i + v_{i+1/2} * dt
# 3. Új gyorsulások kiszámítása (a_{i+1}) az új pozíciókkal
# 4. Sebességek frissítése a maradék fél lépéssel: v_{i+1} = v_{i+1/2} + a_{i+1} * dt/2

# Itt egy egyszerűsített (Euler-Cromer szerű) változatot használunk:
# Először kiszámoljuk az erőket/gyorsulásokat a jelenlegi állapot alapján,
# majd frissítjük a sebességeket és utána a pozíciókat az *új* sebességekkel.
def integrate(pos, vel, mass, forces, dt):
    # Gyorsulás kiszámítása: a = F/m
    # Mivel itt a tömeg egységes, egyszerűsíthetnénk, de tartsuk általánosan
    # Át kell formálni a tömeget a broadcasting művelethez: (N,) -> (N, 1),
    # mert F 2D-s ([m1], [m2], [m3]])
    # Biztonsági ellenőrzés a nullával való osztás ellen (bár a tömeg itt nem nulla)
    # tombok tombje
    mass_reshaped = mass[:, np.newaxis]
    # Kerüljük a nullával való osztást, ha valamelyik tömeg mégis 0 lenne
    safe_mass = np.where(mass_reshaped == 0, 1e-9, mass_reshaped)
    accel = forces / safe_mass

    # Sebességek frissítése
    new_vel = vel + accel * dt

    # Pozíciók frissítése az *új* sebességgel
    new_pos = pos + new_vel * dt

    return new_pos, new_vel


# Vizualizáció
def plot_particles(pos, vel, step, output_dir, padding, k_neighbors):
    plt.style.use('dark_background') # Sötét háttér használata
    fig, ax = plt.subplots(figsize=(10, 10)) # Ábra és tengelyek létrehozása

    # Sűrűség alapú színezés ---
    if N_PARTICLES > k_neighbors: # Csak akkor van értelme, ha van elég részecske
        try:
            # 1. KD-fa építése a pozíciókból a gyors szomszédkereséshez
            kdtree = KDTree(pos)
            # 2. Lekérdezzük minden ponthoz a k+1 legközelebbi szomszéd távolságát
            #    (azért k+1, mert saját maga lesz a legközelebbi)
            # a distances egy tömb, ahol minden sor egy ponthod tartozik,
            # az oszlopok szomszédok távolságait tartalmazzák. A _
            # azt jelenti, hogy a masodik visszatérési értékre (az indexre)
            # most nincs szükségünk
            distances, _ = kdtree.query(pos, k=k_neighbors + 1)
            # 3. Vesszük a k-adik legközelebbi szomszéd távolságát (a 0. a saját maga)
            r_k = distances[:, k_neighbors]
            # 4. Sűrűség becslése: arányos 1/r_k^2 -vel
            # Egy r_k sugarú kör területe pi r_k^2, és ebben van kb. k részecske
            # (k pi r_k^2 \propto 1/r_k^2)
            #    Kerüljük a nullával osztást egy kis epsilon hozzáadásával vagy minimum értékkel
            r_k = np.maximum(r_k, 1e-5) # Minimális távolság a stabilitásért
            density = 1.0 / (r_k**2)
            # 5. Normalizáljuk a sűrűséget a [0, 1] tartományba a színskála számára
            #    Használjunk percentileket a kiugró értékek kezelésére (pl. nagyon sűrű vagy ritka pontok)
            d_min, d_max = np.percentile(density[np.isfinite(density)], [5, 95]) # Csak a véges értékeket nézzük
            norm_density = np.clip((density - d_min) / (d_max - d_min + 1e-9), 0, 1) # Biztonságos osztás
            norm_density[~np.isfinite(norm_density)] = 0 # Kezeljük a NaN/inf értékeket (pl. ha d_max == d_min)

        except Exception as e:
            # Hiba esetén (pl. KDTree hiba, kevés pont) térjünk vissza egyszerű színezésre
            print(f"Hiba a sűrűség számításakor: {e}. Egyszínű ábrázolás.")
            norm_density = np.zeros(N_PARTICLES) # Vagy np.ones, vagy egy konstans szín

    else:
        # Ha nincs elég pont a k-szomszéd alapú sűrűséghez, használjunk egyszínű ábrázolást
        norm_density = np.zeros(N_PARTICLES)

    # colormap kiválasztása (pl. 'plasma', 'viridis', 'coolwarm', 'inferno', 'hot')
    cmap = plt.get_cmap('inferno') # Az 'inferno' vagy 'hot' jól mutat sűrűségnél
    colors = cmap(norm_density)

    # Kis pontméret használata nagy N esetén
    point_size = 1 if N_PARTICLES > 10000 else 5

    # Pont diagram (scatter plot)
    ax.scatter(pos[:, 0], pos[:, 1], s=point_size, c=colors, alpha=0.7, edgecolors='none')

    # Diagram határainak dinamikus beállítása a részecskék szóródása alapján, de maradjon négyzetes
    try:
        # Biztonságosabb max meghatározás, üres tömböt is kezel
        max_range = np.max(np.abs(pos)) if pos.size > 0 else 1.0
        max_range = max(max_range * 1.1, 0.1) # Adjunk hozzá puffert, de legyen minimum méret
    except ValueError: # Ha pl. NaN értékek vannak
        max_range = 1.0 # Alapértelmezett határ
    ax.set_xlim(-max_range, max_range)
    ax.set_ylim(-max_range, max_range)
    ax.set_aspect('equal', adjustable='box') # Négyzetes képarány

    # Cím és tengelyfeliratok hozzáadása
    ax.set_title(f'N-Test Szimuláció (Barnes-Hut) - Lépés: {step} (Szín: Sűrűség)')
    ax.set_xlabel('X Pozíció')
    ax.set_ylabel('Y Pozíció')

    # Tengelyjelölések eltávolítása a tisztább kinézetért
    ax.set_xticks([])
    ax.set_yticks([])

    # Képkocka mentése
    filename = os.path.join(output_dir, f"frame_{step:0{padding}d}.png")
    plt.savefig(filename, dpi=150) # Állítsd a dpi-t a felbontásnak megfelelően
    plt.close(fig) # Ábra bezárása a memória felszabadításához

# Fő Szimulációs Ciklus ---
print("Szimuláció indítása...")
start_time = time.time() # Teljes szimulációs idő mérésének kezdete

for step in range(TIME_STEPS):
    step_start_time = time.time() # Egy lépés idejének mérése

    # 1. QuadTree felépítése az aktuális pozíciók alapján
    root = build_quadtree(pos, mass)

    # 2. Erők kiszámítása minden részecskére a fa segítségével
    forces = get_all_forces(root, pos, mass, THETA)

    # 3. Integrálás (pozíciók és sebességek frissítése)
    pos, vel = integrate(pos, vel, mass, forces, DT)

    # 4. Vizualizáció (minden PLOT_EVERY lépésben)
    if step % PLOT_EVERY == 0:
        # A sűrűség számításhoz szükséges k szomszéd számát átadjuk
        plot_particles(pos, vel, step, OUTPUT_DIR, FRAME_PADDING_DIGITS, K_NEIGHBORS_FOR_DENSITY)
        print(f"Lépés {step}/{TIME_STEPS} kész. Képkocka mentve. (Idő: {time.time() - step_start_time:.2f}s)")

# Teljes szimulációs idő kiszámítása és kiírása
total_time = time.time() - start_time
print(f"Szimuláció befejeződött {total_time:.2f} másodperc alatt.")
print(f"A képkockák a '{OUTPUT_DIR}' könyvtárba lettek mentve. Most összeállíthatod őket videóvá.")

A kimeneti képkockák ide lesznek mentve: nbody_szimulacio_kimenet
Szimuláció indítása...
Lépés 0/90 kész. Képkocka mentve. (Idő: 272.20s)
Lépés 1/90 kész. Képkocka mentve. (Idő: 269.47s)
Lépés 2/90 kész. Képkocka mentve. (Idő: 265.17s)
Lépés 3/90 kész. Képkocka mentve. (Idő: 261.99s)
Lépés 4/90 kész. Képkocka mentve. (Idő: 264.66s)
Lépés 5/90 kész. Képkocka mentve. (Idő: 256.83s)
Lépés 6/90 kész. Képkocka mentve. (Idő: 255.28s)
Lépés 7/90 kész. Képkocka mentve. (Idő: 250.94s)
Lépés 8/90 kész. Képkocka mentve. (Idő: 244.63s)
Lépés 9/90 kész. Képkocka mentve. (Idő: 236.42s)
Lépés 10/90 kész. Képkocka mentve. (Idő: 234.67s)
Lépés 11/90 kész. Képkocka mentve. (Idő: 231.13s)
Lépés 12/90 kész. Képkocka mentve. (Idő: 219.63s)
Lépés 13/90 kész. Képkocka mentve. (Idő: 217.08s)
Lépés 14/90 kész. Képkocka mentve. (Idő: 220.62s)
Lépés 15/90 kész. Képkocka mentve. (Idő: 218.82s)
Lépés 16/90 kész. Képkocka mentve. (Idő: 220.02s)
Lépés 17/90 kész. Képkocka mentve. (Idő: 215.93s)
Lépés 18/90 kész. Kép

KeyboardInterrupt: 

In [ ]:
import imageio, os

frames = sorted(os.listdir('nbody_szimulacio_kimenet'))
video  = [imageio.imread(f'nbody_szimulacio_kimenet/{f}') for f in frames]
imageio.mimsave('nbody_simulation.mp4', video, fps=8)


<ipython-input-6-c7f901ddcbe3>:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  video  = [imageio.imread(f'nbody_szimulacio_kimenet/{f}') for f in frames]
